# Generate a dual-level sampling grid for the world

This notebook generates the grid of square 'patches' used for sampling areas of interest. Each patch has a unique name based on the 'slippygrid' standard used by Google Maps and is considered immutable in the FloodMapper system. The geometry of the patches are stored in the 'world_grid' table of the FloodMapper database and **should not be changed**.

The FloodMapper grid has several layers:

1. A grid of small patches that are used to download and process satellite data.
1. One (or more) grids of larger patches that are used to perform spatial merges before vectorising the results.

For each layer, the slippygrid zoom level is expanded in three steps as a function of latitude to compensate for area distortion of the Web Mercator projection.

Note: the schema of the database is available in the file [floodmapper-db-schema.sql](floodmapper-db-schema.sql).

In [ ]:
# Necessary imports
import os
os.environ['USE_PYGEOS'] = '0'
import itertools
from shapely.geometry import box
import warnings
import numpy as np
import math as math
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import mercantile
import folium
import matplotlib.pyplot as plt
from tqdm import tqdm

from dotenv import load_dotenv
from db_utils import DB
from grid_utils import custom_feature
from grid_utils import gen_zone_patches
from ml4floods.data import utils

# Set bucket will not be requester pays
utils.REQUESTER_PAYS_DEFAULT = False

## Load environment and project details

As with the other notebooks, we load credentials and project details from a hidden ```.env``` file.

In [ ]:
# Load environment variables (including path to credentials) from '.env' file
env_file_path = "../.env"

# Uncomment for alternative version for Windows (r"" indicates raw string)
#env_file_path = r"C:/Users/User/floodmapper/.env"

assert load_dotenv(dotenv_path=env_file_path) == True, "[ERR] Failed to load environment!"
assert "GOOGLE_APPLICATION_CREDENTIALS" in os.environ, "[ERR] Missing $GOOGLE_APPLICATION_CREDENTIAL!"
assert "GS_USER_PROJECT" in os.environ, "[ERR] Missing $GS_USER_PROJECT!"
key_file_path = os.environ["GOOGLE_APPLICATION_CREDENTIALS"]
assert os.path.exists(key_file_path), f"[ERR] Google credential key file does not exist: \n{key_file_path} "
assert "ML4FLOODS_BASE_DIR" in os.environ, "[ERR] Missing $ML4FLOODS_BASE_DIR!"
base_path = os.environ["ML4FLOODS_BASE_DIR"]
assert os.path.exists(base_path), f"[ERR] Base path does not exist: \n{base_path} "
bucket_name = os.environ["BUCKET_URI"]
assert bucket_name is not None and bucket_name != "", f"Bucket name not defined {bucket_name}"
print("[INFO] Successfully loaded FloodMapper environment.")

In [ ]:
# Connect to the database (point to the .env file for credentials)
db_conn = DB(env_file_path)

## User-supplied grid parameters

In [ ]:
# Longitude bounds of total area
# For this test, a strip encompassing Australia
bounds_w = 112.900000000000
bounds_e = 153.63872785102905

# Smallest Zoom level [10]
max_zoom_level = 10

## Generate the zone parameters

Each dataframe containes two 'p_level' zooms.

In [ ]:
# Generate a dataframe for each geographic zone
Z_1_df = gen_zone_patches("1", max_zoom_level, bounds_w, bounds_e)
Z_2S_df = gen_zone_patches("2S", max_zoom_level, bounds_w, bounds_e)
Z_2N_df = gen_zone_patches("2N", max_zoom_level, bounds_w, bounds_e)
Z_3S_df = gen_zone_patches("3S", max_zoom_level, bounds_w, bounds_e)
Z_3N_df = gen_zone_patches("3N", max_zoom_level, bounds_w, bounds_e)

# Merge into a single dataframe
zone_lst = [Z_1_df, Z_2S_df, Z_2N_df, Z_3S_df, Z_3N_df]
grid_gdf = pd.concat(zone_lst)

In [ ]:
m = Z_1_df[Z_1_df.p_level==2].explore(style_kwds={"fillOpacity": 0.1, "color": "magenta"}, name="Z 1", highlight=False)
Z_2S_df[Z_2S_df.p_level==2].explore(style_kwds={"fillOpacity": 0.1, "color": "blue"}, name="Z 2S", highlight=False, m=m)
Z_3S_df[Z_3S_df.p_level==2].explore(style_kwds={"fillOpacity": 0.1, "color": "red"}, name="Z 3S", highlight=False, m=m)
Z_2N_df[Z_2N_df.p_level==2].explore(style_kwds={"fillOpacity": 0.1, "color": "blue"}, name="Z 2N", highlight=False, m=m)
Z_3N_df[Z_3N_df.p_level==2].explore(style_kwds={"fillOpacity": 0.1, "color": "red"}, name="Z 3N", highlight=False, m=m)

folium.LayerControl(collapsed=False).add_to(m)

In [ ]:
m

## Upload the table to the database

Now we run a SQL command to write the 'world_grid' table to the database. This will take a few minutes to complete.

In [ ]:
# Drop all previous rows
query = (f"DELETE FROM world_grid")
db_conn.run_query(query)

In [ ]:
# Insert the new grid entries in batch mode
data = []
print("[INFO] Formatting rows:")
for row in tqdm(grid_gdf.itertuples(), total=len(grid_gdf)):
    data.append(
        (row.patch_name, row.quadkey, row.zoom, row.cent_x, row.cent_y,
         row.cos_factor, row.p_level, row.zone, str(row.geometry))
    )
query = (f"INSERT INTO world_grid"
         f"(patch_name, quadkey, zoom, cent_x, cent_y, "
         f"cos_factor, p_level, zone, geometry) "
         f"VALUES (%s, %s, %s, %s, %s, %s, %s, %s, ST_GeomFromText(%s, 4326)) "
         f"ON CONFLICT (patch_name) DO NOTHING;")
print("[INFO] Inserting table in batch mode.", flush=True)
db_conn.run_batch_insert(query, data)
print("[INFO] DONE")

Once the INSERT query has completed, we can check for a successful upload by querying and plotting the grid.

In [ ]:
# Define and execute the query
query = (f"SELECT patch_name, p_level, zoom, ST_AsText(geometry) "
         f"FROM world_grid " 
         f"WHERE zone = %s;")
data = ("1",)
subgrid_df = db_conn.run_query(query, data, fetch=True)
print(f"[INFO] Returned {len(subgrid_df)} rows.")

In [ ]:
# Format the results into a correct GeoDataFrame
subgrid_df['geometry'] = gpd.GeoSeries.from_wkt(subgrid_df['st_astext'])
subgrid_df.drop(['st_astext'], axis=1, inplace = True)
subgrid_gdf = gpd.GeoDataFrame(subgrid_df, geometry='geometry', crs="EPSG:4326")

In [ ]:
# Plot the Zone 1 grid at two p_levels
m = subgrid_gdf[subgrid_gdf.p_level==1].explore(style_kwds={"fillOpacity": 0.1, "color": "magenta"}, name="P 1", highlight=False)
subgrid_gdf[subgrid_gdf.p_level==2].explore(style_kwds={"fillOpacity": 0.1, "color": "blue"}, name="P 2", highlight=False, m=m)

folium.LayerControl(collapsed=False).add_to(m)

In [ ]:
m